In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [15]:
shop_df = pd.read_csv("shoplocation_v4.csv", encoding='mac_roman')
car_df = pd.read_csv("merged_stoppoints.csv", encoding='mac_roman')

In [16]:
def distance(x1, y1, x2, y2):
    return ((x2 - x1)**2 + (y2 - y1)**2)**0.5

def selectMinDist(locations,clat_center,clong_center):
    if len(locations) >= 2 :
        dist = np.zeros(len(locations))
        for i, location in enumerate(locations):
            # filter the rows where shopname is location
            filtered_df = shop_df.loc[shop_df['shopname'] == location]
            # get the lat_center and long_center values for the filtered rows
            slat_center = filtered_df['lat_center'].iloc[0]
            slong_center = filtered_df['long_center'].iloc[0]
            dist[i] = distance(slat_center,slong_center,clat_center,clong_center)
        min_index = np.argmin(dist)
        return locations[min_index]

In [ ]:
# define a function to get the shop names within the range of a car
alpha = -1.9278469879507876e-18
alpha = 0
def get_shops_within_range(row):
    # get the longitude and latitude center of the car
    clong_center = row['long_center']
    clat_center = row['lat_center']
    
    # filter shop_df to get the shops within the latitude and longitude range of the car
    mask = (shop_df['lat2']-alpha < clat_center) & (shop_df['lat1']+alpha > clat_center) & (shop_df['long1']-alpha < clong_center) & (shop_df['long2']+alpha > clong_center)
    shops_within_range = shop_df.loc[mask, 'shopname'].tolist()
    if len(shops_within_range) >= 2:
        shops_within_range = selectMinDist(shops_within_range,clat_center,clong_center)
    return shops_within_range

# apply the function to car_df and add the result to a new column named 'shop'
car_df['location'] = car_df.apply(get_shops_within_range, axis=1)
car_df['location'] = car_df['location'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

# # print the updated car_df
car_df.to_csv("merged_stoppoints_shop.csv", index=False)
car_df